<a href="https://colab.research.google.com/github/rskrisel/pandas/blob/main/Pandas_Plotly_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python for Public Policy — Session 2 Workbook

**Data Manipulation with pandas + Interactive Charts with Plotly Express**

## Lesson 1: Warm‑Up — Tiny DataFrame

In [6]:
import pandas as pd

toys = {
    'country': ['A', 'B', 'C'],
    'year': [2005, 2006, 2007],
    'arrivals': [10, 0, 5]
}

toy_df = pd.DataFrame(toys)
toy_df

,country,year,arrivals
0,A,2005,10
1,B,2006,0
2,C,2007,5


**Try it**
- Show only the `country` and `arrivals` columns.
- Filter rows where `arrivals` > 0.
- Compute `pct_total` = arrivals / total * 100.

In [7]:
# Your code here
# toy_df[['country', 'arrivals']]
# toy_df[toy_df['arrivals'] > 0]
# toy_df['pct_total'] = toy_df['arrivals'] / toy_df['arrivals'].sum() * 100
# toy_df

## Lesson 2: Load the Refugee Arrivals Dataset

**Data citation**
- U.S. refugee arrivals (2005–2015), compiled from the Refugee Processing Center (via Jeremy Singer‑Vine).

In [8]:
import pandas as pd
url = 'https://raw.githubusercontent.com/GC-DRI/DRI24/main/uploads/pandas/refugee-arrivals-by-destination.csv'
refugee_df = pd.read_csv(url)
len(refugee_df), refugee_df.head(3)

(121245,
    year       origin dest_state dest_city  arrivals
 0  2005  Afghanistan    Alabama    Mobile         0
 1  2006  Afghanistan    Alabama    Mobile         0
 2  2007  Afghanistan    Alabama    Mobile         0)

### Quick exploration

In [9]:
refugee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121245 entries, 0 to 121244
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   year        121245 non-null  int64 
 1   origin      121245 non-null  object
 2   dest_state  121245 non-null  object
 3   dest_city   121245 non-null  object
 4   arrivals    121245 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 4.6+ MB


In [10]:
refugee_df.sample(5)

,year,origin,dest_state,dest_city,arrivals
41607,2007,Afghanistan,Iowa,Des Moines,1
117437,2014,Russia,Washington,Federal Way,4
9651,2006,Iran,California,Panorama City,3
111547,2009,Iraq,Virginia,Potomac Falls,0
116696,2010,Laos,Washington,Bothell,0


**Try it**
- Show the last 7 rows.
- Show the first 10 unique origins.

In [11]:
# refugee_df.tail(7)
# refugee_df['origin'].unique()[:10]

## Lesson 3: Cleaning Basics

### Data types
Keep `year` as an integer for grouping. Add `year_dt` for date operations if needed.

In [12]:
refugee_df['year_dt'] = pd.to_datetime(refugee_df['year'], format='%Y')
refugee_df.dtypes

,0
year,int64
origin,object
dest_state,object
dest_city,object
arrivals,int64
year_dt,datetime64[ns]


### Duplicates

In [13]:
refugee_df[refugee_df.duplicated(keep=False)].head(10)

,year,origin,dest_state,dest_city,arrivals,year_dt
13,2007,Burma,Alabama,Auburn,0,2007-01-01
14,2008,Burma,Alabama,Auburn,0,2008-01-01
15,2009,Burma,Alabama,Auburn,0,2009-01-01
16,2010,Burma,Alabama,Auburn,0,2010-01-01
17,2007,Burma,Alabama,Auburn,0,2007-01-01
18,2008,Burma,Alabama,Auburn,0,2008-01-01
19,2009,Burma,Alabama,Auburn,0,2009-01-01
20,2010,Burma,Alabama,Auburn,0,2010-01-01
155,2013,Cuba,Alabama,Russellville,0,2013-01-01
156,2014,Cuba,Alabama,Russellville,0,2014-01-01


In [14]:
refugee_df = refugee_df.drop_duplicates(keep='first')
refugee_df.shape

(121231, 6)

### Renaming & Selecting

In [15]:
refugee_df = refugee_df.rename(columns={'dest_state':'state', 'dest_city':'city'})
refugee_df.columns.tolist()

['year', 'origin', 'state', 'city', 'arrivals', 'year_dt']

In [16]:
state_only_df = refugee_df[['state']]
state_series = refugee_df['state']
state_only_df.head(2), type(state_only_df), type(state_series)

(     state
 0  Alabama
 1  Alabama,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series)

**Try it**
Using `.loc`, select rows where `origin == 'Iraq'` and columns `['year','state','city','arrivals']`. Then compute the mean of `arrivals`.

In [17]:
# iraq = refugee_df.loc[refugee_df['origin']=='Iraq', ['year','state','city','arrivals']]
# iraq_mean = iraq['arrivals'].mean()
# iraq.head(3), iraq_mean

## Lesson 4: Grouping & Summarizing

In [18]:
by_origin = refugee_df.groupby('origin')
arrivals_sum = by_origin['arrivals'].sum()
arrivals_sum_sorted = arrivals_sum.sort_values(ascending=False)
arrivals_top20 = arrivals_sum_sorted.head(20)
arrivals_top20

,arrivals
origin,
Burma,151875
Iraq,127326
Bhutan,85316
Somalia,71120
Cuba,39946
Iran,36683
Dem. Rep. Congo,25493
Russia,15656
Eritrea,14645


In [19]:
ref_sum_df = arrivals_top20.to_frame().reset_index()
ref_sum_df.head()

,origin,arrivals
0,Burma,151875
1,Iraq,127326
2,Bhutan,85316
3,Somalia,71120
4,Cuba,39946


**Try it**
Compute the top 10 states by total arrivals and save as `state_sum_df`.

In [20]:
# state_sum_df = (refugee_df.groupby('state')['arrivals']
#                                .sum()
#                                .sort_values(ascending=False)
#                                .head(10)
#                                .to_frame()
#                                .reset_index())
# state_sum_df

## Lesson 5: Quick Visuals with pandas `.plot()`

In [21]:
ax = ref_sum_df.plot(kind='bar', x='origin', y='arrivals', figsize=(12,6),
                      title='Total refugee arrivals by country of origin (Top 20)')

**Try it**
Make a line chart of total arrivals by year.

In [22]:
# total_by_year = refugee_df.groupby('year')['arrivals'].sum()
# ax = total_by_year.plot(kind='line', figsize=(10,5), title='Total arrivals by year')

## Lesson 6: Interactive Charts with Plotly Express

In [23]:
# !pip -q install plotly
import plotly.express as px

### Bar chart (interactive)

In [24]:
origin_sum = (refugee_df.groupby('origin')['arrivals']
               .sum().sort_values(ascending=False).head(15).reset_index())
fig = px.bar(origin_sum, x='origin', y='arrivals',
             title='Total refugee arrivals by origin (Top 15)')
fig.show()

### Line chart (interactive)

In [25]:
total_by_year = refugee_df.groupby('year')['arrivals'].sum().reset_index()
fig = px.line(total_by_year, x='year', y='arrivals', markers=True,
              title='Total refugee arrivals by year (2005–2015)')
fig.show()

### Scatter with faceting

In [26]:
focus = refugee_df[refugee_df['origin'].isin(['Iraq','Somalia','Bhutan','Burma','Iran'])]
by_pair = focus.groupby(['origin','year'])['arrivals'].sum().reset_index()
fig = px.scatter(by_pair, x='year', y='arrivals', color='origin',
                 facet_col='origin', facet_col_wrap=3,
                 title='Yearly arrivals for selected origins')
fig.show()

## Save Artifacts (CSV)

In [27]:
refugee_df.to_csv('refugee_clean.csv', index=False)
try:
    from google.colab import files
    files.download('refugee_clean.csv')
except Exception:
    print('File saved to working directory.')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
End of Workbook.